In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor
import warnings
import collections
warnings.filterwarnings("ignore")


# Data Creation
> wml = returns of WinnerMinusLoser portfolio <br>
> iml = returns of IlliquidMinusLiquid portfolio <br>
> market_value = market return <br>

In [20]:
wml_data = pd.read_csv('wml.csv', delimiter=',', header=0)
iml_data = pd.read_csv('iml.csv', delimiter=',', header=0)
target = pd.read_csv('returns.csv', delimiter=',', header=0)
market_value = pd.read_csv('market_value.csv', delimiter=',', header=0)

target = target[213:-1]
target = target.reset_index(drop=True)
data = pd.DataFrame({'iml': iml_data['iml'][213:]})
new_col = pd.DataFrame({'wml': wml_data['wml'][213:]})
data = data.join(new_col)
data = data.reset_index(drop=True)
mv_data = market_value['Adj Close']
new_col = pd.DataFrame({'market_value': mv_data})
data = data.join(new_col)

data.isnull().any()
data = data.dropna()

print(target)

                     Time        AA        AB  ...         Y        ZF       ZTR
0    01-Dec-2008 00:00:00 -0.406048 -0.539028  ... -0.077262 -0.233134 -0.114858
1    01-Jan-2009 00:00:00 -0.680451 -0.663414  ... -0.284478 -0.324837 -0.171092
2    01-Feb-2009 00:00:00 -0.720361 -0.752316  ... -0.375277 -0.469448 -0.325371
3    01-Mar-2009 00:00:00 -0.689459 -0.643621  ... -0.203213 -0.334789 -0.224962
4    01-Apr-2009 00:00:00 -0.781630 -0.725427  ... -0.221628 -0.374591 -0.218945
..                    ...       ...       ...  ...       ...       ...       ...
116  01-Aug-2018 00:00:00  0.309511  0.261939  ... -0.066462 -0.052306 -0.066612
117  01-Sep-2018 00:00:00  0.040853  0.307766  ...  0.038421 -0.066503 -0.058999
118  01-Oct-2018 00:00:00 -0.098241  0.321254  ...  0.153225 -0.077129 -0.024826
119  01-Nov-2018 00:00:00 -0.063522  0.282334  ...  0.138213 -0.076313 -0.019385
120  01-Dec-2018 00:00:00 -0.018732  0.338259  ...  0.123178 -0.082250 -0.064734

[121 rows x 571 columns]


In [21]:
data

,iml,wml,market_value
0,-0.748791,-0.791276,1280.000000
1,-0.527755,-0.565741,1267.380005
2,-0.672975,-0.731377,1282.829956
3,-0.875614,-0.877369,1166.359985
4,-0.788466,-0.778987,968.750000
...,...,...,...
115,0.198859,0.234025,2823.810059
116,0.167494,0.176966,2713.830078
117,0.179813,0.207818,2640.870117
118,0.128124,0.136609,2648.050049


# XGBRegressor model
> X = feature set with names { wml, iml, market_value }<br>
> Target = monthly stock returns<br>
> each stock has its own model<br>
> RMSE - measures the average error performed by the model in predicting the outcome for an observation

In [22]:
import warnings
warnings.filterwarnings("ignore")

# model with random choosen parameters
stocks = target.columns.values
X = data[['iml', 'wml', 'market_value']].values
data_matrixes = []

#create a model for each stock
for Y in stocks[1:]:
    data_dmatrix = xgb.DMatrix(data=X,label=target[Y].values[1:])
    data_dmatrix.feature_names = ['iml', 'wml', 'market_value']
    data_matrixes.append(data_dmatrix)
    X_train, X_test, Y_train, Y_test = train_test_split(X, target[Y].values[1:], test_size=0.1, random_state=0, shuffle=False)
    # store data for finetuning parameters
    if Y == 'ADX':
        X_train_fine = X_train
        Y_train_fine = Y_train
        X_test_fine = X_test
        Y_test_fine = Y_test
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10, n_jobs = 8)
    xg_reg.fit(X_train,Y_train)

    preds = xg_reg.predict(X_test)
    # RMSE - measures the average error performed by the model in predicting the outcome for an observation
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
    print("---- %s ----- RMSE: %f" % (Y, rmse))



---- AA ----- RMSE: 0.278579
---- AB ----- RMSE: 0.083678
---- ABM ----- RMSE: 0.447800
---- ABT ----- RMSE: 0.104342
---- ABX ----- RMSE: 0.423981
---- ADM ----- RMSE: 0.210436
---- ADX ----- RMSE: 0.131121
---- AEM ----- RMSE: 0.307862
---- AEP ----- RMSE: 0.263819
---- AET ----- RMSE: 0.164426
---- AFG ----- RMSE: 0.190463
---- AFL ----- RMSE: 0.085517
---- AGN ----- RMSE: 0.552304
---- AIG ----- RMSE: 0.328393
---- AIN ----- RMSE: 0.101860
---- AIR ----- RMSE: 0.075113
---- AIT ----- RMSE: 0.126762
---- AJG ----- RMSE: 0.091046
---- AJRD ----- RMSE: 0.283694
---- ALE ----- RMSE: 0.227856
---- ALK ----- RMSE: 0.541323
---- ALX ----- RMSE: 0.330312
---- AME ----- RMSE: 0.112715
---- AON ----- RMSE: 0.191951
---- AOS ----- RMSE: 0.249980
---- AP ----- RMSE: 0.602299
---- APA ----- RMSE: 0.263574
---- APB ----- RMSE: 0.201223
---- APC ----- RMSE: 0.357100
---- APD ----- RMSE: 0.118282
---- ARL ----- RMSE: 0.607809
---- ARNC ----- RMSE: 0.438986
---- ARW ----- RMSE: 0.230660
---- ASA --

In [0]:
# function for model measuring
def compare_rmse(train_value, test_value):
  if (abs(test_value - train_value)) < 0.2:
    return "Model is good"
  else: return "Model is OVERFITTED or UNDERFITTED"

# CROSS VALIDATION OF XGBRegressor model
- parameters set randomly
- measuring with RMSE - if train and test values are similar than model is good
- kfold = 5

In [24]:
# cross-validation
for indeks, Y in enumerate(stocks[1:]):
  params = {"objective":"reg:squarederror",'colsample_bytree': 0.8,'learning_rate': 0.1,
                  'max_depth': 5, 'alpha': 10}

  cv_results = xgb.cv(dtrain=data_matrixes[indeks], params=params, nfold=5,
                      num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
  print("------ " + Y + " ------")
  print("Compare RMSE_TRAIN and RMSE_TEST")
  print(cv_results["test-rmse-mean"].iloc[-1])
  print(compare_rmse((cv_results["train-rmse-mean"].iloc[-1]), (cv_results["test-rmse-mean"].iloc[-1])))


------ AA ------
Compare RMSE_TRAIN and RMSE_TEST
0.39443799999999996
Model is good
------ AB ------
Compare RMSE_TRAIN and RMSE_TEST
0.35736819999999997
Model is good
------ ABM ------
Compare RMSE_TRAIN and RMSE_TEST
0.21957140000000003
Model is good
------ ABT ------
Compare RMSE_TRAIN and RMSE_TEST
0.1737718
Model is good
------ ABX ------
Compare RMSE_TRAIN and RMSE_TEST
0.4871616
Model is good
------ ADM ------
Compare RMSE_TRAIN and RMSE_TEST
0.2224116
Model is good
------ ADX ------
Compare RMSE_TRAIN and RMSE_TEST
0.18208200000000002
Model is good
------ AEM ------
Compare RMSE_TRAIN and RMSE_TEST
0.3882116
Model is good
------ AEP ------
Compare RMSE_TRAIN and RMSE_TEST
0.1778932
Model is good
------ AET ------
Compare RMSE_TRAIN and RMSE_TEST
0.2732754
Model is good
------ AFG ------
Compare RMSE_TRAIN and RMSE_TEST
0.20376080000000002
Model is good
------ AFL ------
Compare RMSE_TRAIN and RMSE_TEST
0.2851318
Model is good
------ AGN ------
Compare RMSE_TRAIN and RMSE_TEST
0

# The most important feature in the model


In [25]:
# find the most important feature in model
importance_scores = []
for indeks, Y in enumerate(stocks[1:]):
  xg_reg = xgb.train(params=params, dtrain=data_matrixes[indeks], num_boost_round=10)
  #xgb.plot_tree(xg_reg,num_trees=0)
  #plt.rcParams['figure.figsize'] = [50, 10]
  #plt.show()
  importance_scores.append(xg_reg.get_score())

iml_importance = 0
wml_importance = 0
mv_importance = 0
for score in importance_scores:
  for key in score:
      if(key=='iml'):
        iml_importance+=score[key]
      if(key=='wml'):
        wml_importance+=score[key]
      if(key=='market_value'):
        mv_importance+=score[key]

iml_importance = iml_importance/len(stocks[1:])
wml_importance = wml_importance/len(stocks[1:])
mv_importance = mv_importance/len(stocks[1:])
print("Importance of iml feature: " + str(iml_importance))
print("Importance of wml feature: " + str(wml_importance))
print("Importance of market_value feature: " + str(mv_importance))

Importance of iml feature: 5.2105263157894735
Importance of wml feature: 2.8280701754385964
Importance of market_value feature: 2.0596491228070177


In [26]:
# Tunning the parameters
"""from sklearn.model_selection import RandomizedSearchCV   #Perforing grid search
import warnings
warnings.filterwarnings("ignore")

# first max_depth and min_child_weight --> highest impact on model
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = RandomizedSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_distributions = param_test1, scoring='roc_auc',n_jobs=8,iid=False, cv=5)
gsearch1.fit(X_train_fine, Y_train_fine)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_"""

'from sklearn.model_selection import RandomizedSearchCV   #Perforing grid search\nimport warnings\nwarnings.filterwarnings("ignore")\n\n# first max_depth and min_child_weight --> highest impact on model\nparam_test1 = {\n \'max_depth\':range(3,10,2),\n \'min_child_weight\':range(1,6,2)\n}\ngsearch1 = RandomizedSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=5,\n min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,\n objective= \'reg:squarederror\', nthread=4, scale_pos_weight=1, seed=27), \n param_distributions = param_test1, scoring=\'roc_auc\',n_jobs=8,iid=False, cv=5)\ngsearch1.fit(X_train_fine, Y_train_fine)\ngsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_'